### Problem

What vulnerable entities fall within an individual dams inundation zone (output in a list, showing the different entities for a selected dam)
Count how many vulnerable entities are within a dam's inundation zone (just a number)

In [61]:
import psycopg2
from tabulate import tabulate
from psycopg2 import extensions


In [62]:
db_params = {
    "dbname": "gisdb",
    "user": "admin",
    "password": "admin",
    "host": "localhost",
    "port": 5432,
}

try:
    conn = psycopg2.connect(**db_params)
    print("Connected to the database successfully.")
except Exception as e:
    print(f"Error connecting to the database: {e}")

Connected to the database successfully.


In [63]:
def ensure_conn_ready(conn):
    if conn.get_transaction_status() != extensions.TRANSACTION_STATUS_IDLE:
        conn.rollback()

In [64]:
def get_actual_column_names(conn, table, schema="public"):
    ensure_conn_ready(conn)

    """Return the case-sensitive column names for schema.table"""
    with conn.cursor() as cur:
        cur.execute(
            """
            SELECT column_name
            FROM information_schema.columns
            WHERE table_schema = %s AND table_name = %s
            """,
            (schema, table),
        )
        return [r[0] for r in cur.fetchall()]

In [65]:
def list_available_dams(conn, limit=50):
    """Return dams sorted by inundation-polygon area (largest first)."""
    with conn.cursor() as cur:
        dam_cols = get_actual_column_names(conn, "utah_dams")
        name_col = next(c for c in dam_cols if c.upper() == "NAME")
        type_col = next(c for c in dam_cols if c.upper() == "TYPE")
        geom_col = "geom"

        cur.execute(
            f"""
            SELECT "{name_col}", "{type_col}",
                   ST_Area("{geom_col}"::geography) / 1e6 AS area_sq_km
            FROM utah_dams
            ORDER BY area_sq_km DESC
            LIMIT %s
            """,
            (limit,),
        )
        rows = cur.fetchall()

    headers = ["Dam Name", "Type", "Inundation Area (sq km)"]
    print("\nTop", limit, "Utah dams by inundation area:")
    print(tabulate([[r[0], r[1], round(r[2], 2)] for r in rows], headers, tablefmt="grid"))
    return rows  # [(name, type, area)]

In [66]:
def analyze_power_plants_at_risk(conn, dam_name):
    """Print and return the power plants whose points fall within *dam_name* polygon."""
    with conn.cursor() as cur:
        # Column discovery – dams
        dam_cols = get_actual_column_names(conn, "utah_dams")
        name_col_dam = next(c for c in dam_cols if c.upper() == "NAME")
        geom_col_dam = "geom"

        # Column discovery – power plants
        pp_cols = get_actual_column_names(conn, "power_plants")
        name_col_pp = next(c for c in pp_cols if c.upper() == "NAME")
        type_col_pp = next(c for c in pp_cols if c.upper() == "TYPE")
        fuel_col = next(c for c in pp_cols if c.upper() == "PRIM_FUEL")
        cap_col = next(c for c in pp_cols if c.upper() == "SUMMER_CAP")
        geom_col_pp = "geom"

        # Spatial query – intersects rather than contains (safer for points)
        cur.execute(
            f"""
            SELECT p."{name_col_pp}", p."{type_col_pp}", p."{fuel_col}", p."{cap_col}"
            FROM   utah_dams d
            JOIN   power_plants p
              ON   ST_Intersects(d."{geom_col_dam}", p."{geom_col_pp}")
            WHERE  d."{name_col_dam}" = %s
            ORDER  BY p."{cap_col}" DESC NULLS LAST
            """,
            (dam_name,),
        )
        rows = cur.fetchall()

    print("\n--- Power Plants at Risk ---")
    if rows:
        headers = ["Plant Name", "Type", "Primary Fuel", "Capacity (MW)"]
        print(tabulate(rows, headers, tablefmt="grid"))
    else:
        print(f"No power plants found within the inundation zone of '{dam_name}'.")
    return rows

In [70]:
""" 
This analyze_power_plants_at_risk_v2() is corrected version of v1 above.

Unions all dam parts for that NAME (avoids duplicates)

Builds plant points from lat/lon (robust when geom is missing)

Returns the dam name using the parameter instead of referencing td.NAME

Makes PRIM_FUEL optional (if not present, it returns NULL)

Rolls back on error so you can continue
"""
def analyze_power_plants_at_risk_v2(conn, dam_name):
    ensure_conn_ready(conn)

    """Print and return unique power plants whose points fall within dam_name's inundation geometry."""
    try:
        with conn.cursor() as cur:
            # Discover column names
            dam_cols = get_actual_column_names(conn, "utah_dams")
            name_col_dam = next(c for c in dam_cols if c.upper() == "NAME")
            geom_col_dam = "geom"  # change if different

            pp_cols = get_actual_column_names(conn, "power_plants")
            name_col_pp = next(c for c in pp_cols if c.upper() == "NAME")
            type_col_pp = next(c for c in pp_cols if c.upper() == "TYPE")
            cap_col     = next(c for c in pp_cols if c.upper() == "SUMMER_CAP")

            # PRIM_FUEL might not exist; make it optional
            fuel_col = next((c for c in pp_cols if c.upper() == "PRIM_FUEL"), None)

            cur.execute(
                f"""
                WITH target_dam AS (
                    SELECT ST_Union(d."{geom_col_dam}") AS geom
                    FROM utah_dams d
                    WHERE d."{name_col_dam}" = %s
                ),
                plant_geoms AS (
                    SELECT
                        id,
                        ST_SetSRID(ST_MakePoint("LONGITUDE","LATITUDE"),4326) AS geom,
                        "{name_col_pp}" AS plant_name,
                        "{type_col_pp}" AS plant_type,
                        {"\"" + fuel_col + "\"" if fuel_col else "NULL"} AS primary_fuel,
                        "{cap_col}" AS capacity_mw
                    FROM power_plants
                    WHERE "LATITUDE" IS NOT NULL AND "LONGITUDE" IS NOT NULL
                )
                SELECT DISTINCT ON (pg.id)
                       pg.plant_name,
                       pg.plant_type,
                       pg.primary_fuel,
                       pg.capacity_mw,
                       %s AS dam_name
                FROM target_dam td
                JOIN plant_geoms pg
                  ON ST_Intersects(td.geom, pg.geom)
                ORDER BY pg.id, pg.capacity_mw DESC NULLS LAST;
                """,
                (dam_name, dam_name),
            )
            rows = cur.fetchall()

        # Pretty print
        print("\n--- Power Plants at Risk (unique) ---")
        if rows:
            headers = ["Plant Name", "Type", "Primary Fuel", "Capacity (MW)", "Dam Name"]
            try:
                from tabulate import tabulate
                print(tabulate(rows, headers, tablefmt="grid"))
            except Exception:
                print(headers)
                for r in rows:
                    print(r)
        else:
            print(f"No power plants found within the inundation zone of '{dam_name}'.")
        return rows

    except Exception as e:
        # Roll back the transaction so future queries work
        conn.rollback()
        raise


In [71]:
dams = list_available_dams(conn)
print("\nEnter a dam name to analyze or type 'ALL' for every dam")


Top 50 Utah dams by inundation area:
+--------------------------------------+-----------+---------------------------+
| Dam Name                             | Type      |   Inundation Area (sq km) |
+======================================+===========+===========================+
| SEVIER BRIDGE                        | PMF       |                    863.32 |
+--------------------------------------+-----------+---------------------------+
| SEVIER BRIDGE                        | Sunny     |                    808.46 |
+--------------------------------------+-----------+---------------------------+
| DMAD                                 | PMF       |                    581.88 |
+--------------------------------------+-----------+---------------------------+
| PIUTE                                | PMF       |                    224.4  |
+--------------------------------------+-----------+---------------------------+
| SETTLEMENT CANYON                    | PMF       |                   

In [47]:
selected = input("Dam name: ").strip()
if not selected:
    selected = dams[0][0]  # largest dam by area

if selected.upper() == "ALL":
    dam_names = [d[0] for d in dams]
else:
    dam_names = [selected]

Dam name:  ALL


In [55]:
conn.rollback()   # reset the failed transaction state

In [69]:
for dam in dam_names:
    print("\n" + "=" * 60)
    print(f"POWER-PLANT RISK REPORT FOR {dam}")
    print("=" * 60)
    plants = analyze_power_plants_at_risk_v2(conn, dam)
    print(f"\nTotal plants at risk: {len(plants)}\n")


POWER-PLANT RISK REPORT FOR SEVIER BRIDGE

--- Power Plants at Risk (unique) ---
No power plants found within the inundation zone of 'SEVIER BRIDGE'.

Total plants at risk: 0


POWER-PLANT RISK REPORT FOR SEVIER BRIDGE

--- Power Plants at Risk (unique) ---
No power plants found within the inundation zone of 'SEVIER BRIDGE'.

Total plants at risk: 0


POWER-PLANT RISK REPORT FOR DMAD

--- Power Plants at Risk (unique) ---
No power plants found within the inundation zone of 'DMAD'.

Total plants at risk: 0


POWER-PLANT RISK REPORT FOR PIUTE

--- Power Plants at Risk (unique) ---
No power plants found within the inundation zone of 'PIUTE'.

Total plants at risk: 0


POWER-PLANT RISK REPORT FOR SETTLEMENT CANYON

--- Power Plants at Risk (unique) ---
No power plants found within the inundation zone of 'SETTLEMENT CANYON'.

Total plants at risk: 0


POWER-PLANT RISK REPORT FOR NEWCASTLE

--- Power Plants at Risk (unique) ---
No power plants found within the inundation zone of 'NEWCASTLE'

In [9]:
conn.close()

# Visualize dam geometries reading from database table

In [12]:
import os
import folium
from folium import plugins
import psycopg2
import json
from IPython.display import IFrame


# Database connection parameters
conn_params = {
    "dbname": "gisdb",
    "user": "admin",
    "password": "admin",
    "host": "localhost",
    "port": 5432
}

# Create the map centered on Utah
m = folium.Map(location=[40.7607, -111.8939], zoom_start=7)

# Connect to PostGIS
conn = psycopg2.connect(**conn_params)
cur = conn.cursor()

# Fetch each MultiPolygon feature as GeoJSON
cur.execute("""
    SELECT
        id,
        "NAME",
        "Shape__Area",
        ST_AsGeoJSON(geom) AS geojson
    FROM utah_dams;
""")

# Add each polygon to map
for row in cur.fetchall():
    dam_id, name, area_m2, geom_json = row
    geojson = json.loads(geom_json)

    # Compute area in square kilometers
    area_km2 = float(area_m2) / 1_000_000 if area_m2 else 0.0

    popup_html = f"""
        <strong>Dam Name:</strong> {name or "Unknown"}<br>
        <strong>Area:</strong> {area_km2:.2f} km²
    """

    folium.GeoJson(
        data=geojson,
        name=f"Dam {dam_id}",
        style_function=lambda x: {
            "fillColor": "blue",
            "color": "black",
            "weight": 1,
            "fillOpacity": 0.4
        },
        highlight_function=lambda x: {"weight": 2, "color": "yellow"},
        tooltip=folium.Tooltip(name),
        popup=folium.Popup(popup_html, max_width=300)
    ).add_to(m)

# Add fullscreen button
plugins.Fullscreen(
    position="topright",
    title="Full Screen",
    title_cancel="Exit Full Screen",
    force_separate_button=True
).add_to(m)

# Save and show map

m.save("dams_inundation_map.html")

map_path = os.path.abspath("dams_inundation_map.html")
IFrame(src=map_path, width=800, height=600)

# Cleanup
cur.close()
conn.close()


# Power plants visualization from database table

In [13]:
import folium
from folium import plugins
import psycopg2

# DB connection parameters
conn_params = {
    "dbname": "gisdb",
    "user": "admin",
    "password": "admin",
    "host": "localhost",
    "port": 5432
}

# Create map
m = folium.Map(location=[40.7607, -111.8939], zoom_start=6)

# Connect to PostGIS
conn = psycopg2.connect(**conn_params)
cur = conn.cursor()

# Query: Get lat/lon and power plant name
cur.execute("""
    SELECT "LATITUDE", "LONGITUDE", "NAME"
    FROM power_plants
    WHERE "LATITUDE" IS NOT NULL AND "LONGITUDE" IS NOT NULL;
""")

# Plot markers
for lat, lon, name in cur.fetchall():
    folium.Circle(
        location=[lat, lon],
        radius=10,
        fill=True,
        fill_opacity=0.9,
        color='blue',
        popup=folium.Popup(name or "Unknown", max_width=200)
    ).add_to(m)

# Add fullscreen button
plugins.Fullscreen(
    position="topright",
    title="Full Screen",
    title_cancel="Exit Full Screen",
    force_separate_button=True
).add_to(m)

# Save to HTML
m.save("power_plants_map.html")
m

# Cleanup
cur.close()
conn.close()
